In [2]:
import bs4
from bs4 import BeautifulSoup
import datetime
import time
import pandas as pd
import lxml
import html
import pip._vendor.requests as requests
import html5lib

In [3]:
pd.set_option("display.max.columns",19)

Web Scraping Basketball stats. Namely the Standings table for the 22/23 regular season, Point per Game of Members of the Teams and Full RG Schedule of each team

In [4]:
url = "https://www.basketball-reference.com/leagues/NBA_2023_standings.html" #Website
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"}

In [5]:
page = requests.get(url, headers=headers) #Request access to the HTML of the site 
soup = BeautifulSoup(page.content, "html.parser") #Use BS4 to parse the HTML, making it understandable

Standings= pd.read_html(page.content, match='Table') #Read in the Standings table. There are many tables
Standings #I want just the first two tables which are Eastern and Western conference standings

[          Eastern Conference   W   L   W/L%    GB   PS/G   PA/G   SRS
 0       Milwaukee Bucks* (1)  58  24  0.707     —  116.9  113.3  3.61
 1        Boston Celtics* (2)  57  25  0.695   1.0  117.9  111.4  6.38
 2    Philadelphia 76ers* (3)  54  28  0.659   4.0  115.2  110.9  4.37
 3   Cleveland Cavaliers* (4)  51  31  0.622   7.0  112.3  106.9  5.23
 4       New York Knicks* (5)  47  35  0.573  11.0  116.0  113.1  2.99
 5         Brooklyn Nets* (6)  45  37  0.549  13.0  113.4  112.5  1.03
 6            Miami Heat* (7)  44  38  0.537  14.0  109.5  109.8 -0.13
 7         Atlanta Hawks* (8)  41  41  0.500  17.0  118.4  118.1  0.32
 8       Toronto Raptors* (9)  41  41  0.500  17.0  112.9  111.4  1.59
 9        Chicago Bulls* (10)  40  42  0.488  18.0  113.1  111.8  1.37
 10       Indiana Pacers (11)  35  47  0.427  23.0  116.3  119.5 -2.91
 11   Washington Wizards (12)  35  47  0.427  23.0  113.2  114.4 -1.06
 12        Orlando Magic (13)  34  48  0.415  24.0  111.4  114.0 -2.39
 13   

In [6]:
standings_east_conf = Standings[0] #First Table
standings_east_conf

,Eastern Conference,W,L,W/L%,GB,PS/G,PA/G,SRS
0,Milwaukee Bucks* (1),58,24,0.707,—,116.9,113.3,3.61
1,Boston Celtics* (2),57,25,0.695,1.0,117.9,111.4,6.38
2,Philadelphia 76ers* (3),54,28,0.659,4.0,115.2,110.9,4.37
3,Cleveland Cavaliers* (4),51,31,0.622,7.0,112.3,106.9,5.23
4,New York Knicks* (5),47,35,0.573,11.0,116.0,113.1,2.99
5,Brooklyn Nets* (6),45,37,0.549,13.0,113.4,112.5,1.03
6,Miami Heat* (7),44,38,0.537,14.0,109.5,109.8,-0.13
7,Atlanta Hawks* (8),41,41,0.500,17.0,118.4,118.1,0.32
8,Toronto Raptors* (9),41,41,0.500,17.0,112.9,111.4,1.59
9,Chicago Bulls* (10),40,42,0.488,18.0,113.1,111.8,1.37


In [7]:
standings_west_conf = Standings[1] #2nd Table
standings_west_conf


,Western Conference,W,L,W/L%,GB,PS/G,PA/G,SRS
0,Denver Nuggets* (1),53,29,0.646,—,115.8,112.5,3.04
1,Memphis Grizzlies* (2),51,31,0.622,2.0,116.9,113.0,3.60
2,Sacramento Kings* (3),48,34,0.585,5.0,120.7,118.1,2.30
3,Phoenix Suns* (4),45,37,0.549,8.0,113.6,111.6,2.08
4,Los Angeles Clippers* (5),44,38,0.537,9.0,113.6,113.1,0.31
5,Golden State Warriors* (6),44,38,0.537,9.0,118.9,117.1,1.66
6,Los Angeles Lakers* (7),43,39,0.524,10.0,117.2,116.6,0.43
7,Minnesota Timberwolves* (8),42,40,0.512,11.0,115.8,115.8,-0.22
8,New Orleans Pelicans* (9),42,40,0.512,11.0,114.4,112.5,1.63
9,Oklahoma City Thunder* (10),40,42,0.488,13.0,117.5,116.4,0.96


Lets Clean it Up a little before we continue Scraping

In [8]:
#Now lets rename the columns Eastern Conf, and Western Conf to 'Team Name' so we can concat both tables
standings_east_conf.rename(columns={'Eastern Conference':'Team Name'},inplace=True)

standings_west_conf.rename(columns={'Western Conference':'Team Name'},inplace=True)

#Join both tables as standings table
standings_table = pd.concat([standings_east_conf,standings_west_conf], ignore_index=True,sort=False)
standings_table

,Team Name,W,L,W/L%,GB,PS/G,PA/G,SRS
0,Milwaukee Bucks* (1),58,24,0.707,—,116.9,113.3,3.61
1,Boston Celtics* (2),57,25,0.695,1.0,117.9,111.4,6.38
2,Philadelphia 76ers* (3),54,28,0.659,4.0,115.2,110.9,4.37
3,Cleveland Cavaliers* (4),51,31,0.622,7.0,112.3,106.9,5.23
4,New York Knicks* (5),47,35,0.573,11.0,116.0,113.1,2.99
5,Brooklyn Nets* (6),45,37,0.549,13.0,113.4,112.5,1.03
6,Miami Heat* (7),44,38,0.537,14.0,109.5,109.8,-0.13
7,Atlanta Hawks* (8),41,41,0.500,17.0,118.4,118.1,0.32
8,Toronto Raptors* (9),41,41,0.500,17.0,112.9,111.4,1.59
9,Chicago Bulls* (10),40,42,0.488,18.0,113.1,111.8,1.37


In [9]:
#Clean standings table
standings_table=standings_table.drop(columns="GB") #Drop GB not needed
#Lets make the columns more understandable
standings_table = standings_table.rename(columns=
                    {"PS/G":"Points_Per_Game", "PA/G":"Opp PPG", "SRS":"Simple Rating System"})

In [10]:
#Now lets add a column to signify teams that made the Postseason
postseason_status = ["Yes" 
                  if "*" in p 
                  else "No" 
                  for p in standings_table["Team Name"]]
standings_table["Playoff Status"] = postseason_status


In [11]:
standings_table["Team Name"] = standings_table["Team Name"].str.replace("*", "")#Remove astericks
standings_table[["Team_Name", "num"]] = standings_table["Team Name"].str.split(
    "(", expand=True)#split the Team name column to remove symbols.Name the split columns
standings_table = standings_table.drop(columns=["num","Team Name"])#Drop old Team name column and symbols column


In [12]:
standings_table = standings_table.sort_values(by="W/L%", ascending=False)#Arrange standings by W/%
#Rearrange Table columns
standings_table = standings_table[['Team_Name', 'W', 'L', 'W/L%',
                                  'Points_Per_Game', 'Opp PPG', 'Simple Rating System','Playoff Status']]
standings_table.reset_index(drop = True, inplace=True)#reset index
standings_table.index = standings_table.index + 1 #Let it start from 1
standings_table.index =standings_table.index.set_names(["Ranking"])#Name the index Rank
standings_table

,Team_Name,W,L,W/L%,Points_Per_Game,Opp PPG,Simple Rating System,Playoff Status
Ranking,,,,,,,,
1,Milwaukee Bucks,58,24,0.707,116.9,113.3,3.61,Yes
2,Boston Celtics,57,25,0.695,117.9,111.4,6.38,Yes
3,Philadelphia 76ers,54,28,0.659,115.2,110.9,4.37,Yes
4,Denver Nuggets,53,29,0.646,115.8,112.5,3.04,Yes
5,Cleveland Cavaliers,51,31,0.622,112.3,106.9,5.23,Yes
6,Memphis Grizzlies,51,31,0.622,116.9,113.0,3.60,Yes
7,Sacramento Kings,48,34,0.585,120.7,118.1,2.30,Yes
8,New York Knicks,47,35,0.573,116.0,113.1,2.99,Yes
9,Phoenix Suns,45,37,0.549,113.6,111.6,2.08,Yes


In [13]:
standings_table.rename(columns={"Playoff Status":"Postseason_status"}, inplace=True)

Back To Scraping


Lets get all the Team URLs to access their rosters and Schedules


In [14]:

tableeast = soup.select('table.sortable')[0] #Lets select the Eastern Conference Table
tablewest = soup.select('table.sortable')[1] #Lets select the Western Conference Table
time.sleep(5)
tableeast

<table class="suppress_all sortable stats_table" data-cols-to-freeze=",1" id="confs_standings_E">
<caption>Conference Standings Table</caption>
<colgroup><col/><col/><col/><col/><col/><col/><col/><col/></colgroup>
<thead>
<tr>
<th aria-label="Eastern Conference" class="poptip sort_default_asc left" data-stat="team_name" scope="col">Eastern Conference</th>
<th aria-label="Wins" class="poptip right" data-stat="wins" data-tip="Wins" scope="col">W</th>
<th aria-label="Losses" class="poptip right" data-stat="losses" data-tip="Losses" scope="col">L</th>
<th aria-label="Win-Loss Percentage" class="poptip right" data-stat="win_loss_pct" data-tip="Win-Loss Percentage" scope="col">W/L%</th>
<th aria-label="GB" class="poptip sort_default_asc right" data-stat="gb" data-tip="Games Behind" scope="col">GB</th>
<th aria-label="Points Per Game" class="poptip right" data-stat="pts_per_g" data-tip="Points Per Game" scope="col">PS/G</th>
<th aria-label="Opponent Points Per Game" class="poptip right" data-

In [15]:
 #Lets find all the <a element in the Tables as they contain all the Team links
link1 =tableeast.find_all('a')
link2 =tablewest.find_all('a')

time.sleep(5)

In [16]:
#Lets get all the links

#Eastern Conference first
href1 = [l.get("href") for l in link1]

easturl = [f"https://www.basketball-reference.com{h}" for h in href1]

#Western Conference next
href2 = [l.get("href") for l in link2]
westurl = [f"https://www.basketball-reference.com{h}" for h in href2]

time.sleep(10)

In [17]:
#Add both east and west urls and make it all teams url
teamurl = easturl + westurl
teamurl

['https://www.basketball-reference.com/teams/MIL/2023.html',
 'https://www.basketball-reference.com/teams/BOS/2023.html',
 'https://www.basketball-reference.com/teams/PHI/2023.html',
 'https://www.basketball-reference.com/teams/CLE/2023.html',
 'https://www.basketball-reference.com/teams/NYK/2023.html',
 'https://www.basketball-reference.com/teams/BRK/2023.html',
 'https://www.basketball-reference.com/teams/MIA/2023.html',
 'https://www.basketball-reference.com/teams/ATL/2023.html',
 'https://www.basketball-reference.com/teams/TOR/2023.html',
 'https://www.basketball-reference.com/teams/CHI/2023.html',
 'https://www.basketball-reference.com/teams/IND/2023.html',
 'https://www.basketball-reference.com/teams/WAS/2023.html',
 'https://www.basketball-reference.com/teams/ORL/2023.html',
 'https://www.basketball-reference.com/teams/CHO/2023.html',
 'https://www.basketball-reference.com/teams/DET/2023.html',
 'https://www.basketball-reference.com/teams/DEN/2023.html',
 'https://www.basketball

Test on 1 team first


In [18]:
#Lets use the Team links and get Roster Details. Lets get from the bucks
easturlbucks = easturl[0]
data = requests.get(easturlbucks, headers=headers)

In [19]:
soup = BeautifulSoup(data.content, "html.parser")

In [20]:
#Lets get their schedule
soup = BeautifulSoup(data.content, "html.parser")
links = soup.select('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and "_games" in l]
links = links[0]

scheduledata = requests.get(f"https://www.basketball-reference.com{links}") #Format and make a full URL

#Lets import the Schedule table
scheduletable = pd.read_html(scheduledata.content, match="Regular Season")
scheduletable=scheduletable[0] #To make it a pandas series


time.sleep(4)

In [21]:
#Lets also get the Regular season pergame stats of each member of the team
Rosterinfo = pd.read_html(data.content, match="Per Game") 
roster = Rosterinfo[0]  #To make it a pandas series
roster



,Rk,Player,Age,G,GS,MP,FG,FGA,FG%,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1,Jrue Holiday,32,67,65,32.6,7.3,15.3,0.479,...,1.2,3.9,5.1,7.4,1.2,0.4,2.9,1.7,19.3
1,2,Giannis Antetokounmpo,28,63,63,32.1,11.2,20.3,0.553,...,2.2,9.6,11.8,5.7,0.8,0.8,3.9,3.1,31.1
2,3,Brook Lopez,34,78,78,30.4,6.1,11.5,0.531,...,2.0,4.7,6.7,1.3,0.5,2.5,1.4,2.6,15.9
3,4,Grayson Allen,27,72,70,27.4,3.4,7.7,0.440,...,0.8,2.4,3.3,2.3,0.9,0.2,1.0,1.6,10.4
4,5,Bobby Portis,27,70,22,26.0,5.7,11.5,0.496,...,2.2,7.4,9.6,1.5,0.4,0.2,1.2,1.6,14.1
5,6,Khris Middleton,31,33,19,24.3,5.4,12.3,0.436,...,0.8,3.4,4.2,4.9,0.7,0.2,2.2,2.1,15.1
6,7,Pat Connaughton,30,61,33,23.7,2.7,6.9,0.392,...,0.8,3.8,4.6,1.3,0.6,0.2,0.5,1.1,7.6
7,8,Joe Ingles,35,46,0,22.7,2.3,5.4,0.435,...,0.3,2.5,2.8,3.3,0.7,0.1,1.2,1.6,6.9
8,9,Jevon Carter,27,81,39,22.3,3.0,7.0,0.423,...,0.4,2.1,2.5,2.4,0.8,0.4,1.0,2.0,8.0
9,10,George Hill,36,35,0,19.1,1.7,3.8,0.447,...,0.4,1.5,1.9,2.5,0.5,0.1,0.8,1.2,5.0


IT WORKED!! SO LETS DO THAT FOR ALL TEAMS


In [22]:
nba_roster = []
team_schedule = []

In [23]:
#All Rosters

for i, s in enumerate(teamurl):
    teamname = [item.split('/')[-2] for item in teamurl]
    data = requests.get(s, headers=headers)
    soup = BeautifulSoup(data.content, "html.parser")
    all_roster = pd.read_html(data.content, match="Per Game")[0]

    #All Schedules
    soup = BeautifulSoup(data.content, "html.parser")
    links = [l.get("href") for l in soup.find_all('a')]
    links = [l for l in links if l and "_games" in l]
    scheduledata = requests.get(f"https://www.basketball-reference.com{links[0]}") #Format and make a full URL
    allscheduletable = pd.read_html(scheduledata.content, match="Regular Season")[0]


    allscheduletable['Team'] = teamname[i] #add team name so we can know what team
    all_roster['Team'] = teamname[i]

    team_schedule.append(allscheduletable)
    nba_roster.append(all_roster)

    time.sleep(10)

In [24]:
schedule_df = pd.concat(team_schedule)
roster_df = pd.concat(nba_roster)

In [25]:
roster_df.sort_values(by = ["PTS"], ascending=False)

,Rk,Player,Age,G,GS,MP,FG,FGA,FG%,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Team
1,2,Joel Embiid,28,66,66,34.6,11.0,20.1,0.548,...,8.4,10.2,4.2,1.0,1.7,3.4,3.1,33.1,PHI
1,2,Luka Dončić,23,66,66,36.2,10.9,22.0,0.496,...,7.8,8.6,8.0,1.4,0.5,3.6,2.5,32.4,DAL
0,1,Damian Lillard,32,58,58,36.3,9.6,20.7,0.463,...,4.0,4.8,7.3,0.9,0.3,3.3,1.9,32.2,POR
0,1,Shai Gilgeous-Alexander,24,68,68,35.5,10.4,20.3,0.510,...,4.0,4.8,5.5,1.6,1.0,2.8,2.8,31.4,OKC
1,2,Giannis Antetokounmpo,28,63,63,32.1,11.2,20.3,0.553,...,9.6,11.8,5.7,0.8,0.8,3.9,3.1,31.1,MIL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,21,Sterling Brown,27,4,0,6.0,0.0,1.0,0.000,...,1.3,2.0,0.5,0.8,0.0,0.0,1.0,0.0,LAL
23,24,Alondes Williams,23,1,0,5.0,0.0,0.0,NaN,...,1.0,1.0,0.0,0.0,0.0,2.0,1.0,0.0,BRK
24,25,Moses Brown,23,2,0,3.0,0.0,0.0,NaN,...,0.0,0.0,0.0,0.5,0.0,0.5,0.5,0.0,BRK
19,20,Julian Champagnie,21,2,0,3.5,0.0,1.0,0.000,...,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,PHI


In [26]:
schedule_df

,G,Date,Start (ET),Unnamed: 3,Unnamed: 4,Unnamed: 5,Opponent,Unnamed: 7,Unnamed: 8,Tm,Opp,W,L,Streak,Notes,Team
0,1,"Thu, Oct 20, 2022",7:30p,NaN,Box Score,@,Philadelphia 76ers,W,NaN,90,88,1,0,W 1,NaN,MIL
1,2,"Sat, Oct 22, 2022",8:00p,NaN,Box Score,NaN,Houston Rockets,W,NaN,125,105,2,0,W 2,NaN,MIL
2,3,"Wed, Oct 26, 2022",7:30p,NaN,Box Score,NaN,Brooklyn Nets,W,NaN,110,99,3,0,W 3,NaN,MIL
3,4,"Fri, Oct 28, 2022",8:00p,NaN,Box Score,NaN,New York Knicks,W,NaN,119,108,4,0,W 4,NaN,MIL
4,5,"Sat, Oct 29, 2022",8:00p,NaN,Box Score,NaN,Atlanta Hawks,W,NaN,123,115,5,0,W 5,NaN,MIL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,79,"Tue, Apr 4, 2023",10:00p,NaN,Box Score,@,Phoenix Suns,L,NaN,94,115,20,59,L 1,NaN,SAS
82,80,"Thu, Apr 6, 2023",8:00p,NaN,Box Score,NaN,Portland Trail Blazers,W,NaN,129,127,21,59,W 1,NaN,SAS
83,G,Date,Start (ET),NaN,NaN,NaN,Opponent,NaN,NaN,Tm,Opp,W,L,Streak,Notes,SAS
84,81,"Sat, Apr 8, 2023",4:00p,NaN,Box Score,NaN,Minnesota Timberwolves,L,NaN,131,151,21,60,L 1,NaN,SAS


A LOT OF CLEANING NEEDED BEFORE VIZ



In [27]:
#Index is all over the place. Fix
schedule_df.reset_index(drop=True)


,G,Date,Start (ET),Unnamed: 3,Unnamed: 4,Unnamed: 5,Opponent,Unnamed: 7,Unnamed: 8,Tm,Opp,W,L,Streak,Notes,Team
0,1,"Thu, Oct 20, 2022",7:30p,NaN,Box Score,@,Philadelphia 76ers,W,NaN,90,88,1,0,W 1,NaN,MIL
1,2,"Sat, Oct 22, 2022",8:00p,NaN,Box Score,NaN,Houston Rockets,W,NaN,125,105,2,0,W 2,NaN,MIL
2,3,"Wed, Oct 26, 2022",7:30p,NaN,Box Score,NaN,Brooklyn Nets,W,NaN,110,99,3,0,W 3,NaN,MIL
3,4,"Fri, Oct 28, 2022",8:00p,NaN,Box Score,NaN,New York Knicks,W,NaN,119,108,4,0,W 4,NaN,MIL
4,5,"Sat, Oct 29, 2022",8:00p,NaN,Box Score,NaN,Atlanta Hawks,W,NaN,123,115,5,0,W 5,NaN,MIL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2587,79,"Tue, Apr 4, 2023",10:00p,NaN,Box Score,@,Phoenix Suns,L,NaN,94,115,20,59,L 1,NaN,SAS
2588,80,"Thu, Apr 6, 2023",8:00p,NaN,Box Score,NaN,Portland Trail Blazers,W,NaN,129,127,21,59,W 1,NaN,SAS
2589,G,Date,Start (ET),NaN,NaN,NaN,Opponent,NaN,NaN,Tm,Opp,W,L,Streak,Notes,SAS
2590,81,"Sat, Apr 8, 2023",4:00p,NaN,Box Score,NaN,Minnesota Timberwolves,L,NaN,131,151,21,60,L 1,NaN,SAS


In [28]:
schedule_df = schedule_df[schedule_df["Date"] != "Date"] #This drops the rows that are repeated headings

schedule_df

,G,Date,Start (ET),Unnamed: 3,Unnamed: 4,Unnamed: 5,Opponent,Unnamed: 7,Unnamed: 8,Tm,Opp,W,L,Streak,Notes,Team
0,1,"Thu, Oct 20, 2022",7:30p,NaN,Box Score,@,Philadelphia 76ers,W,NaN,90,88,1,0,W 1,NaN,MIL
1,2,"Sat, Oct 22, 2022",8:00p,NaN,Box Score,NaN,Houston Rockets,W,NaN,125,105,2,0,W 2,NaN,MIL
2,3,"Wed, Oct 26, 2022",7:30p,NaN,Box Score,NaN,Brooklyn Nets,W,NaN,110,99,3,0,W 3,NaN,MIL
3,4,"Fri, Oct 28, 2022",8:00p,NaN,Box Score,NaN,New York Knicks,W,NaN,119,108,4,0,W 4,NaN,MIL
4,5,"Sat, Oct 29, 2022",8:00p,NaN,Box Score,NaN,Atlanta Hawks,W,NaN,123,115,5,0,W 5,NaN,MIL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,78,"Sun, Apr 2, 2023",6:00p,NaN,Box Score,@,Sacramento Kings,W,OT,142,134,20,58,W 1,NaN,SAS
81,79,"Tue, Apr 4, 2023",10:00p,NaN,Box Score,@,Phoenix Suns,L,NaN,94,115,20,59,L 1,NaN,SAS
82,80,"Thu, Apr 6, 2023",8:00p,NaN,Box Score,NaN,Portland Trail Blazers,W,NaN,129,127,21,59,W 1,NaN,SAS
84,81,"Sat, Apr 8, 2023",4:00p,NaN,Box Score,NaN,Minnesota Timberwolves,L,NaN,131,151,21,60,L 1,NaN,SAS


In [29]:
schedule_df['Date'] = pd.to_datetime(schedule_df['Date'], format="%a, %b %d, %Y") 
schedule_df['Date'].dtype

C:\Users\USER\AppData\Local\Temp\ipykernel_19180\2902290466.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  schedule_df['Date'] = pd.to_datetime(schedule_df['Date'], format="%a, %b %d, %Y")


dtype('<M8[ns]')

In [30]:
schedule_df["Day_of_the_week"]=schedule_df['Date'].dt.day_name()
schedule_df["Day_of_the_week"]

C:\Users\USER\AppData\Local\Temp\ipykernel_19180\1652400485.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  schedule_df["Day_of_the_week"]=schedule_df['Date'].dt.day_name()


0      Thursday
1      Saturday
2     Wednesday
3        Friday
4      Saturday
        ...    
80       Sunday
81      Tuesday
82     Thursday
84     Saturday
85       Sunday
Name: Day_of_the_week, Length: 2472, dtype: object

In [31]:
schedule_df= schedule_df.drop(columns=['Unnamed: 4', 'Notes',"Unnamed: 3"])

In [32]:
schedule_df = schedule_df.rename(columns={"Unnamed: 7":"Win or Loss",
                            "Unnamed: 8":"RegularTime/OT",
                            "Tm":"Team_points",
                            "Opp":"Opp_points"}) #Adds more context to column names

In [33]:
#Unnamed 5 looks like a column that tells us if the team is Home or way. So lets make it clearer
schedule_df["Unnamed: 5"] = schedule_df["Unnamed: 5"].str.strip().fillna(
    'Home').replace("@", "Away") 

In [34]:
schedule_df = schedule_df.rename(columns={"Unnamed: 5":"Home/Away"}) #Rename column

In [35]:
schedule_df.reset_index(drop=True)
schedule_df["RegularTime/OT"] = schedule_df["RegularTime/OT"].fillna("RT")

In [36]:
schedule_df["Start (ET)"] = pd.to_datetime(schedule_df["Start (ET)"],format="mixed").dt.strftime("%H:%M")

In [37]:
schedule_df.set_index(["G"],inplace=True)


In [38]:
schedule_df


,Date,Start (ET),Home/Away,Opponent,Win or Loss,RegularTime/OT,Team_points,Opp_points,W,L,Streak,Team,Day_of_the_week
G,,,,,,,,,,,,,
1,2022-10-20,19:30,Away,Philadelphia 76ers,W,RT,90,88,1,0,W 1,MIL,Thursday
2,2022-10-22,20:00,Home,Houston Rockets,W,RT,125,105,2,0,W 2,MIL,Saturday
3,2022-10-26,19:30,Home,Brooklyn Nets,W,RT,110,99,3,0,W 3,MIL,Wednesday
4,2022-10-28,20:00,Home,New York Knicks,W,RT,119,108,4,0,W 4,MIL,Friday
5,2022-10-29,20:00,Home,Atlanta Hawks,W,RT,123,115,5,0,W 5,MIL,Saturday
...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,2023-04-02,18:00,Away,Sacramento Kings,W,OT,142,134,20,58,W 1,SAS,Sunday
79,2023-04-04,22:00,Away,Phoenix Suns,L,RT,94,115,20,59,L 1,SAS,Tuesday
80,2023-04-06,20:00,Home,Portland Trail Blazers,W,RT,129,127,21,59,W 1,SAS,Thursday


In [39]:
roster_df

,Rk,Player,Age,G,GS,MP,FG,FGA,FG%,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Team
0,1,Jrue Holiday,32,67,65,32.6,7.3,15.3,0.479,...,3.9,5.1,7.4,1.2,0.4,2.9,1.7,19.3,MIL
1,2,Giannis Antetokounmpo,28,63,63,32.1,11.2,20.3,0.553,...,9.6,11.8,5.7,0.8,0.8,3.9,3.1,31.1,MIL
2,3,Brook Lopez,34,78,78,30.4,6.1,11.5,0.531,...,4.7,6.7,1.3,0.5,2.5,1.4,2.6,15.9,MIL
3,4,Grayson Allen,27,72,70,27.4,3.4,7.7,0.440,...,2.4,3.3,2.3,0.9,0.2,1.0,1.6,10.4,MIL
4,5,Bobby Portis,27,70,22,26.0,5.7,11.5,0.496,...,7.4,9.6,1.5,0.4,0.2,1.2,1.6,14.1,MIL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18,19,Charles Bassey,22,35,2,14.5,2.5,3.9,0.644,...,3.4,5.5,1.3,0.5,0.9,1.2,2.1,5.7,SAS
19,20,Gorgui Dieng,33,31,1,11.5,1.5,3.8,0.385,...,2.6,3.5,1.7,0.1,0.5,1.0,1.8,3.9,SAS
20,21,Isaiah Roby,24,42,2,11.3,1.6,3.7,0.432,...,2.0,2.5,0.9,0.4,0.2,0.7,1.3,4.1,SAS
21,22,Jordan Hall,21,9,0,9.2,1.0,3.1,0.321,...,1.0,1.3,1.2,0.1,0.0,0.8,0.7,3.1,SAS


Get the tables in CSV format

In [40]:
import os
output_path = os.path.join(os.getcwd(), "output.csv")

In [41]:
standings_table.to_csv("nbastandings.csv",index=False)
roster_df.to_csv("nbaroster.csv",index=False)
schedule_df.to_csv("nbaschedule.csv",index=False)